In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM
from tqdm import tqdm
import os
import gc
from collections import defaultdict
from typing import List, Dict, Any, Optional, Tuple


In [ ]:
# from huggingface_hub import login
# login()  

In [17]:
DEVICE = torch.device('cpu')#'cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH = "meta-llama/Llama-2-7b-chat-hf"
print(f"Using device: {DEVICE}")

Using device: cpu


In [18]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

In [19]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map=DEVICE
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
   

In [20]:
# class ExecutionTracer:
#     def __init__(self):
#         self.execution_order = []
#         self.counter = 0
    
#     def __call__(self, module, input, output):
#         self.counter += 1
#         module_name = None
#         for name, mod in model.named_modules():
#             if mod is module:
#                 module_name = name
#                 break
        
#         self.execution_order.append({
#             'order': self.counter,
#             'name': module_name,
#             'type': module.__class__.__name__,
#             'input_shape': input[0].shape if isinstance(input, tuple) and len(input) > 0 else 'special',
#             'output_shape': output.shape if hasattr(output, 'shape') else type(output).__name__
#         })

# # Create tracer and register hooks
# tracer = ExecutionTracer()
# hooks = []
# for name, module in model.named_modules():
#     # Skip container modules
#     if len(list(module.children())) == 0:
#         hooks.append(module.register_forward_hook(tracer))

# # Run a forward pass
# input_ids = tokenizer("Hello world", return_tensors="pt").input_ids
# with torch.no_grad():
#     output = model(input_ids)

# # Print execution order
# print("LAYER EXECUTION ORDER:")
# print("="*100)
# for item in tracer.execution_order:
#     print(f"{item['order']:3d}. {item['name']:<50} | {item['type']:<20} | {item['input_shape']} → {item['output_shape']}")

# # Clean up hooks
# for hook in hooks:
#     hook.remove()


In [21]:
# =============================================================================
# Distance Metrics for Comparing Logit Distributions
# =============================================================================

def compute_l2_distance(original_logits: torch.Tensor, perturbed_logits: torch.Tensor) -> float:
    # Compute L2 (Euclidean) distance between two logit vectors
    return torch.norm(original_logits - perturbed_logits, p=2).item()

def compute_cosine_distance(original_logits: torch.Tensor, perturbed_logits: torch.Tensor) -> float:
    # Compute cosine distance (1 - cosine_similarity) between two logit vectors
    cos_sim = F.cosine_similarity(original_logits.unsqueeze(0), perturbed_logits.unsqueeze(0))
    return (1 - cos_sim).item()

def compute_kl_divergence(original_logits: torch.Tensor, perturbed_logits: torch.Tensor) -> float:
    # Compute KL divergence: KL(original || perturbed) after softmax
    original_probs = F.softmax(original_logits, dim=-1)
    perturbed_log_probs = F.log_softmax(perturbed_logits, dim=-1)
    # KL(P || Q) = sum(P * log(P/Q)) = sum(P * (log_P - log_Q))
    kl_div = F.kl_div(perturbed_log_probs, original_probs, reduction='sum')
    return kl_div.item()

def compute_js_divergence(original_logits: torch.Tensor, perturbed_logits: torch.Tensor) -> float:
   # Compute Jensen-Shannon divergence: 0.5*KL(P||M) + 0.5*KL(Q||M) where M = 0.5*(P+Q).
    P = F.softmax(original_logits, dim=-1)
    Q = F.softmax(perturbed_logits, dim=-1)
    M = 0.5 * (P + Q)
    
    # KL(P || M)
    kl_pm = F.kl_div(M.log(), P, reduction='sum')
    # KL(Q || M)
    kl_qm = F.kl_div(M.log(), Q, reduction='sum')
    
    js_div = 0.5 * (kl_pm + kl_qm)
    return js_div.item()

def compute_all_distances(original_logits: torch.Tensor, perturbed_logits: torch.Tensor) -> Dict[str, float]:
    # Compute all distance metrics between original and perturbed logits.
    return {
        'l2_distance': compute_l2_distance(original_logits, perturbed_logits),
        'cosine_distance': compute_cosine_distance(original_logits, perturbed_logits),
        'kl_divergence': compute_kl_divergence(original_logits, perturbed_logits),
        'js_divergence': compute_js_divergence(original_logits, perturbed_logits),
    }

In [22]:
# Global variables for detailed activation capture
captured_activations = {}
current_hooks = []
hook_errors = []

def clear_activations():
    global captured_activations
    captured_activations.clear()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def remove_all_hooks():
    global current_hooks
    for hook in current_hooks:
        try:
            hook.remove()
        except:
            pass
    current_hooks.clear()

def get_activation_hook(name):
    def hook(module, input, output):
        global hook_errors
        try:
            # Handle different output types
            if output is None:
                activation = None
            elif isinstance(output, tuple):
                activation = output[0]
            elif hasattr(output, 'last_hidden_state'):
                # Handle model output objects
                activation = output.last_hidden_state
            else:
                activation = output
            
            # Handle input
            input_tensor = input[0] if isinstance(input, tuple) and len(input) > 0 else None

            # Safely detach and move to CPU
            def safe_detach_cpu(tensor):
                if tensor is None:
                    return None
                try:
                    # Check if tensor is on meta device
                    if hasattr(tensor, 'device') and str(tensor.device) == 'meta':
                        return None
                    return tensor.detach().cpu()
                except Exception as e:
                    hook_errors.append(f"Detach error in {name}: {str(e)}")
                    return None

            captured_activations[name] = {
                'output': safe_detach_cpu(activation),
                'input': safe_detach_cpu(input_tensor),
                'weight': safe_detach_cpu(module.weight) if hasattr(module, 'weight') and module.weight is not None else None,
                'bias': safe_detach_cpu(module.bias) if hasattr(module, 'bias') and module.bias is not None else None
            }
        except Exception as e:
            error_msg = f"Hook error in {name}: {str(e)}"
            hook_errors.append(error_msg)
            captured_activations[name] = {'output': None, 'input': None, 'weight': None, 'bias': None}
    return hook

def register_llama_hooks(model):
    global current_hooks
    remove_all_hooks() # clear any old hooks first
    hook_errors.clear()

    total_layers = len(model.model.layers)

    for i in range(total_layers):
        layer = model.model.layers[i]
        layer_prefix = f"layer_{i}"
        components = [
            (layer.self_attn.q_proj, f"{layer_prefix}_attention_q"), (layer.self_attn.k_proj, f"{layer_prefix}_attention_k"),
            (layer.self_attn.v_proj, f"{layer_prefix}_attention_v"), (layer.self_attn.o_proj, f"{layer_prefix}_attention_output"),
            (layer.mlp.gate_proj, f"{layer_prefix}_mlp_gate"), (layer.mlp.up_proj, f"{layer_prefix}_mlp_up"),
            (layer.mlp.down_proj, f"{layer_prefix}_mlp_down"), (layer.input_layernorm, f"{layer_prefix}_input_norm"),
            (layer.post_attention_layernorm, f"{layer_prefix}_post_attn_norm"),
        ]
        for module, name in components:
            current_hooks.append(module.register_forward_hook(get_activation_hook(name)))
    
    current_hooks.append(model.model.norm.register_forward_hook(get_activation_hook("final_norm")))
    current_hooks.append(model.lm_head.register_forward_hook(get_activation_hook("lm_head")))
    print(f"Registered {len(current_hooks)} hooks.")

def run_model_and_capture_activations(model, inputs=None, inputs_embeds=None):
    global hook_errors
    clear_activations()
    register_llama_hooks(model)
    
    with torch.no_grad():
        if inputs is not None:
            _ = model(**inputs)
        elif inputs_embeds is not None:
            _ = model(inputs_embeds=inputs_embeds)
        else:
            raise ValueError("Either inputs or inputs_embeds must be provided.")
            
    remove_all_hooks()
    
    # Print any hook errors that occurred
    if hook_errors:
        print(f"WARNING: {len(hook_errors)} hook errors occurred:")
        for err in hook_errors[:5]:
            print(f"  - {err}")
        if len(hook_errors) > 5:
            print(f"  ... and {len(hook_errors) - 5} more")
    
    # return a copy of the captured activations
    return captured_activations.copy()

In [23]:
# Neuron Perturbation Sensitivity Analysis

def get_top_k_predictions(logits: torch.Tensor, tokenizer, k: int = 3) -> Dict[str, Any]:
    probs = F.softmax(logits, dim=-1)
    top_logits, top_indices = torch.topk(logits, k)
    top_probs = probs[top_indices]
    
    result = {}
    for i in range(k):
        idx = top_indices[i].item()
        word = tokenizer.decode([idx])
        result[f'top{i+1}_word'] = word
        result[f'top{i+1}_index'] = idx
        result[f'top{i+1}_logit'] = top_logits[i].item()
        result[f'top{i+1}_softmax'] = top_probs[i].item()
    
    return result

def analyze_neuron_perturbation_sensitivity(
    model: "LlamaForCausalLM",
    tokenizer: "LlamaTokenizer",
    final_norm_activations: torch.Tensor,
    original_logits: torch.Tensor,
    k_values: List[int],
    perturbation_deltas: List[float] = None,
    num_trials_per_k: int = 10,
) -> List[Dict[str, Any]]:
   
    if perturbation_deltas is None:
        perturbation_deltas = [0.1]
    
    results = []
    
    # Get dimensions
    seq_len = final_norm_activations.shape[1]
    hidden_size = final_norm_activations.shape[2] 
    last_token_pos = seq_len - 1
    
    # Get original logits for the last token
    original_last_logits = original_logits[0, last_token_pos, :].float()
    
    # Get top-3 predictions for original logits (same for all trials)
    original_top3 = get_top_k_predictions(original_last_logits, tokenizer, k=3)
    # Prefix with 'orig_' to distinguish from perturbed
    original_top3_prefixed = {f'orig_{k}': v for k, v in original_top3.items()}
    
    # Get lm_head weights (and bias if present)
    lm_head_weight = model.lm_head.weight.detach()
    lm_head_bias = None
    if hasattr(model.lm_head, 'bias') and model.lm_head.bias is not None:
        lm_head_bias = model.lm_head.bias.detach()
    
    # Iterate over all combinations of K and delta
    for delta in perturbation_deltas:
        for k in k_values:
            if k > hidden_size:
                print(f"Warning: K={k} exceeds hidden_size={hidden_size}, skipping.")
                continue
                
            for trial in range(num_trials_per_k):
                # Clone the activations
                perturbed_activations = final_norm_activations.clone()
                
                # Randomly select K neuron indices to perturb 
                neuron_indices = torch.randperm(hidden_size)[:k].tolist()
                
                # Apply perturbation: add delta to selected neurons
                for idx in neuron_indices:
                    perturbed_activations[0, last_token_pos, idx] += delta
                
                # Compute perturbed logits through lm_head 
                perturbed_last_activations = perturbed_activations[0, last_token_pos, :].to(lm_head_weight.device)
                perturbed_logits = F.linear(perturbed_last_activations, lm_head_weight, lm_head_bias).float()
                
                # Compute all distance metrics
                distances = compute_all_distances(original_last_logits.to(perturbed_logits.device), perturbed_logits)
                
                # Get top-3 predictions for perturbed logits
                perturbed_top3 = get_top_k_predictions(perturbed_logits, tokenizer, k=3)
                # Prefix with 'pert_' to distinguish from original
                perturbed_top3_prefixed = {f'pert_{k}': v for k, v in perturbed_top3.items()}
                
                # Record results
                result = {
                    'num_neurons_changed': k,
                    'trial': trial,
                    'perturbation_value': delta,
                    'neuron_indices': str(neuron_indices),  
                    'total_neurons': hidden_size,
                    'soundness_ratio': k / hidden_size,
                    **distances,
                    **original_top3_prefixed,
                    **perturbed_top3_prefixed,
                }
                results.append(result)
    
    return results

In [ ]:
def save_perturbation_results(
    results_list: List[Dict[str, Any]],
    input_text: str,
    input_id: int,
    filename: str = "neuron_perturbation_analysis.csv"
):
    # Save perturbation analysis results to CSV.
    if not results_list:
        print("No results to save.")
        return
        
    df = pd.DataFrame(results_list)
    df.insert(0, 'input_id', input_id)
    #df.insert(0, 'input_text', input_text)
    
    # Append to the file if it exists, otherwise create it
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)
    
    print(f"--- Saved {len(df)} perturbation analysis rows to {filename} ---")

In [25]:
# =============================================================================
# Main Workflow: Neuron Perturbation Sensitivity Analysis
# =============================================================================

def run_perturbation_analysis_workflow(
    model: "LlamaForCausalLM",
    tokenizer: "LlamaTokenizer",
    string_input: List,  # [input_id, input_text]
    k_values: List[int] = None,
    perturbation_deltas: List[float] = None,
    num_trials_per_k: int = 10,
):

    input_id, input_text = string_input
    sample_input = tokenizer(input_text, return_tensors="pt")
    inputs_on_device = {k: v.to(model.device) for k, v in sample_input.items()}
    
    print(f"\n{'='*60}")
    print(f"Input ID: {input_id}")
    print(f"Input: '{tokenizer.decode(inputs_on_device['input_ids'][0])}'")
    print(f"K values to test: {k_values}")
    print(f"Perturbation deltas: {perturbation_deltas}")
    print(f"Trials per (K, delta): {num_trials_per_k}")
    print(f"Total experiments: {len(k_values) * len(perturbation_deltas) * num_trials_per_k}")
    print(f"{'='*60}")
    
    # --- Step 1: Get Original Logits and Activations ---
    print("Running forward pass to capture original state...")
    
    with torch.no_grad():
        original_logits = model(**inputs_on_device).logits
    
    # Capture activations (we only need final_norm)
    original_activations = run_model_and_capture_activations(model, inputs=inputs_on_device)
    
    # Get final_norm output (layer n-1, input to lm_head)
    try:
        final_norm_output = original_activations['final_norm']['output'].to(model.device)
    except KeyError:
        print("ERROR: Could not find 'final_norm' in activations.")
        return
    
    hidden_size = final_norm_output.shape[2]
    print(f"Final norm output shape: {final_norm_output.shape}")
    print(f"Hidden size (N = total neurons): {hidden_size}")
    
    # Show original prediction
    last_token_logits = original_logits[0, -1, :]
    top_token_idx = torch.argmax(last_token_logits).item()
    print(f"Original top prediction: '{tokenizer.decode(top_token_idx)}' (ID: {top_token_idx})")
    
    # --- Step 2: Run Perturbation Analysis ---
    print(f"\nRunning perturbation analysis...")
    
    results = analyze_neuron_perturbation_sensitivity(
        model=model,
        tokenizer=tokenizer,
        final_norm_activations=final_norm_output,
        original_logits=original_logits,
        k_values=k_values,
        perturbation_deltas=perturbation_deltas,
        num_trials_per_k=num_trials_per_k,
    )
    
    # --- Step 3: Save Results ---
    save_perturbation_results(results, input_text, input_id)
    
    # --- Step 4: Print Summary ---
    print(f"\n--- Summary for Input {input_id} ---")
    df = pd.DataFrame(results)
    # Group by both perturbation_value and num_neurons_changed
    summary = df.groupby(['perturbation_value', 'num_neurons_changed']).agg({
        'l2_distance': ['mean', 'std'],
        'cosine_distance': ['mean', 'std'],
        'kl_divergence': ['mean', 'std'],
        'js_divergence': ['mean', 'std'],
        'soundness_ratio': 'first'
    }).round(6)
    print(summary)
    
    # Clean up
    del original_activations, final_norm_output, results
    clear_activations()
    
    print(f"\n{'='*60}")
    print("Analysis complete.")
    print(f"{'='*60}")

In [26]:
sample_texts = [
    [1,"The capital of France is"],
    [2,"The largest mammal on Earth is"],
    [3,"The process of photosynthesis occurs in"],
    [4,"The speed of light in a vacuum is"],
    [5,"The chemical symbol for gold is"],
    [6,"The human body has how many bones"],
    [7,"The Great Wall of China was built to"],
    [8,"Water boils at what temperature"],
    [9,"The smallest unit of matter is"],
    [10,"Shakespeare wrote the play"],
    [11,"The currency of Japan is"],
    [12,"Mount Everest is located in"],
    [13,"The inventor of the telephone was"],
    [14,"DNA stands for"],
    [15,"The largest ocean on Earth is"],
    [16,"The planet closest to the Sun is"],
    [17,"Gravity was discovered by"],
    [18,"The Amazon rainforest is primarily located in"],
    [19,"The freezing point of water is"],
    [20,"The most abundant gas in Earth's atmosphere is"],
    [21,"The Mona Lisa was painted by"],
    [22,"The longest river in the world is"],
    [23,"Photosynthesis converts carbon dioxide and water into"],
    [24,"The study of earthquakes is called"],
    [25,"The first person to walk on the moon was"]
]

In [ ]:
# =============================================================================
# Run Perturbation Analysis on All Sample Texts
# =============================================================================

# Configuration
K_VALUES = [i for i in range(1, 1025)] # Number of neurons to perturb
PERTURBATION_DELTAS = [i*0.1 for i in range(-100, 101)]  # Multiple delta values to test
NUM_TRIALS_PER_K = [5000]  # Number of random trials per (K, delta) combination

# Loop through each prompt
for i, prompt in enumerate(sample_texts):
    print(f"\n>>>> Starting Perturbation Analysis for Prompt {i+1}/{len(sample_texts)} <<<<")
    
    run_perturbation_analysis_workflow(
        model=model,
        tokenizer=tokenizer,
        string_input=prompt,
        k_values=K_VALUES,
        perturbation_deltas=PERTURBATION_DELTAS,
        num_trials_per_k=NUM_TRIALS_PER_K,
    )

print("\n\n<<<< ALL PERTURBATION ANALYSES COMPLETE >>>>")
print(f"Results saved to 'neuron_perturbation_analysis.csv'")


>>>> Starting Analysis for Prompt 1 (Mode: min) <<<<

Input: '<s> The capital of France is'
Registered 290 hooks.
--- Logit Swap Attack ---
Original top prediction: 'Paris' (ID: 3681)
Target swap token:     'textt' (ID: 16196)
New top prediction after swap: 'a'


--- [Recon 1/2] Starting reconstruction ---


Optimizing Recon 1:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# # =============================================================================
# # Quick Test: Run on a single input to verify output
# # =============================================================================

# # Test with just the first prompt, fewer K values, and multiple deltas
# TEST_PROMPT = [1, "The capital of France is"]
# TEST_K_VALUES = [1, 4, 16, 64]  
# TEST_DELTAS = [110]  
# TEST_TRIALS = 3  

# print("Running quick test with multiple deltas...")
# run_perturbation_analysis_workflow(
#     model=model,
#     tokenizer=tokenizer,
#     string_input=TEST_PROMPT,
#     k_values=TEST_K_VALUES,
#     perturbation_deltas=TEST_DELTAS,
#     num_trials_per_k=TEST_TRIALS,
# )
# print("\nTest complete! Check 'neuron_perturbation_analysis.csv' for output.")


Running quick test with multiple deltas...

Input ID: 1
Input: '<s> The capital of France is'
K values to test: [1, 4, 16, 64]
Perturbation deltas: [110]
Trials per (K, delta): 3
Total experiments: 12
Running forward pass to capture original state...
Registered 290 hooks.
Final norm output shape: torch.Size([1, 6, 4096])
Hidden size (N = total neurons): 4096
Original top prediction: 'Paris' (ID: 3681)

Running perturbation analysis...
--- Saved 12 perturbation analysis rows to neuron_perturbation_analysis.csv ---

--- Summary for Input 1 ---
                                        l2_distance             \
                                               mean        std   
perturbation_value num_neurons_changed                           
110                1                     317.804759   9.647455   
                   4                     662.068156  32.622029   
                   16                   1342.695435  50.912951   
                   64                   2603.869548  35.